## Welcome to our Chatbot "MEDANT 1.0"
It is a semi context-aware medical assistant. Let's go through it.
<img src="https://icon-library.net/images/bot-icon/bot-icon-6.jpg" width="500" height="400">


## In order to build the Chatbot, we need some helper libraries and frameworks
### Let's import them

In [1]:
import numpy as np
import tensorflow as tf
import joblib
import matplotlib.pyplot as plt
import pickle
from tensorflow.keras import layers , activations , models , preprocessing
import json
import random
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

In [2]:
questions = list()
answers = list()
answers_with_tags = list()

In [3]:
with open('FullData.json') as js:
    data = json.load(js)
    print(type(data))
    for i in data:
        
            
        questions.append(i['patient'])
        answers_with_tags.append(i['agent'])
    

<class 'list'>


In [4]:
len(questions)

6574

In [5]:
len(answers_with_tags)

6574

In [6]:
from tensorflow.keras import preprocessing , utils

for i in range( len( answers_with_tags ) ) :
    answers.append( '<START> ' + answers_with_tags[i] + ' <END>' )

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( questions + answers )
VOCAB_SIZE = len( tokenizer.word_index )+1
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))


VOCAB SIZE : 553


In [7]:
dic=tokenizer.word_index

In [8]:
list(dic.keys())

['i',
 'start',
 'end',
 'a',
 'you',
 'pain',
 'having',
 'is',
 'more',
 'me',
 'am',
 'have',
 'son',
 'the',
 'high',
 'weakness',
 'doctor',
 'to',
 'my',
 'your',
 'abdominal',
 'please',
 'fever',
 'typhoid',
 'tell',
 'that',
 'apetite',
 'muscle',
 'fatigue',
 'and',
 'dengue',
 'jaundice',
 'from',
 'suffering',
 'yellow',
 'be',
 'may',
 'in',
 'daughter',
 'has',
 'for',
 'information',
 'kindly',
 'are',
 'there',
 'but',
 'constipation',
 'headache',
 'poor',
 'stomach',
 'rash',
 'sweating',
 'aches',
 'of',
 'very',
 'urine',
 'loss',
 'chance',
 'skin',
 'get',
 'vomit',
 'certainly',
 'eye',
 'yellowish',
 'pale',
 'stools',
 'joint',
 'eyes',
 'nose',
 'recommend',
 'sorry',
 'would',
 'severe',
 'headaches',
 'vomiting',
 'bleeding',
 'apologies',
 'quite',
 'last',
 'days',
 'thanks',
 'lot',
 'her',
 'about',
 'few',
 'need',
 'thank',
 'him',
 'seem',
 'help',
 'bit',
 'mild',
 'could',
 'with',
 'situation',
 'possibly',
 'whites',
 'region',
 'sudden',
 'probab

In [9]:
tokenizer

In [10]:

# encoder_input_data
tokenized_questions = tokenizer.texts_to_sequences( questions )
maxlen_questions = max( [ len(x) for x in tokenized_questions ] )
padded_questions = preprocessing.sequence.pad_sequences( tokenized_questions , maxlen=maxlen_questions , padding='post' )
encoder_input_data = np.array( padded_questions )
print( encoder_input_data.shape , maxlen_questions )

# decoder_input_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
maxlen_answers = max( [ len(x) for x in tokenized_answers ] )
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
decoder_input_data = np.array( padded_answers )
print( decoder_input_data.shape , maxlen_answers )

# decoder_output_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
onehot_answers = utils.to_categorical( padded_answers , VOCAB_SIZE )
decoder_output_data = np.array( onehot_answers )
print( decoder_output_data.shape )

# Saving all the arrays to storage
np.save( 'enc_in_data.npy' , encoder_input_data )
np.save( 'dec_in_data.npy' , decoder_input_data )
np.save( 'dec_tar_data.npy' , decoder_output_data )


(6574, 27) 27
(6574, 68) 68
(6574, 68, 553)


In [11]:
dim=553  

In [12]:
# new edited
encoder_inputs = tf.keras.layers.Input(shape=( None , ))
encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, dim , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( dim , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( None ,  ))
decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, dim , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( dim , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )




W1217 08:05:30.605082  1936 deprecation.py:506] From D:\Anaconda\envs\tfGPUclone\lib\site-packages\tensorflow\python\keras\initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1217 08:05:30.782609  1936 deprecation.py:506] From D:\Anaconda\envs\tfGPUclone\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1217 08:05:31.121701  1936 deprecation.py:323] From D:\Anaconda\envs\tfGPUclone\lib\site-packages\tensorflow\python\keras\backend.py:3794: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.arr

In [13]:
losses = ['categorical_crossentropy','sparse_categorical_crossentropy','kullback_leibler_divergence','poisson','cosine_proximity']
opts = [tf.keras.optimizers.RMSprop(),tf.keras.optimizers.Adagrad()]

In [14]:
model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 553)    305809      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 553)    305809      input_2[0][0]                    
______________________________________________________________________________________________

In [15]:
model

In [16]:
model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=150, epochs=80 ) 
model.save( 'model3.h5' ) 

Epoch 1/80
6574/6574 [==============================] - 63s 10ms/sample - loss: 0.8776
Epoch 2/80
6574/6574 [==============================] - 45s 7ms/sample - loss: 0.3758
Epoch 3/80
6574/6574 [==============================] - 47s 7ms/sample - loss: 0.1894
Epoch 4/80
6574/6574 [==============================] - 47s 7ms/sample - loss: 0.1237
Epoch 5/80
6574/6574 [==============================] - 47s 7ms/sample - loss: 0.0887
Epoch 6/80
6574/6574 [==============================] - 47s 7ms/sample - loss: 0.0661
Epoch 7/80
6574/6574 [==============================] - 47s 7ms/sample - loss: 0.0565
Epoch 8/80
6574/6574 [==============================] - 47s 7ms/sample - loss: 0.0509
Epoch 9/80
6574/6574 [==============================] - 47s 7ms/sample - loss: 0.0474
Epoch 10/80
6574/6574 [==============================] - 47s 7ms/sample - loss: 0.0460
Epoch 11/80
6574/6574 [==============================] - 47s 7ms/sample - loss: 0.0428
Epoch 12/80
6574/6574 [============================

In [17]:

def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( dim ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( dim ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model


In [18]:

def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        tokens_list.append( tokenizer.word_index[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_questions , padding='post')


In [19]:
enc_model , dec_model = make_inference_models()



In [20]:
def preprocess(sent):
    sent = nltk.word_tokenize(sent)
    sent = nltk.pos_tag(sent)
    return sent

def solve(ex):
    
    sent = preprocess(ex)
    pattern = 'NP: {<DT>?<JJ>*<NN>}'
    cp = nltk.RegexpParser(pattern)
    cs = cp.parse(sent)
    #print(cs)
    from nltk.chunk import conlltags2tree, tree2conlltags
    from pprint import pprint
    iob_tagged = tree2conlltags(cs)
    #pprint(iob_tagged)
    ne_tree = nltk.ne_chunk(pos_tag(word_tokenize(ex)))
    a=str(ne_tree)
    s=""
    n=a.find("PERSON")
    i=n+7
    if n!=-1:
        while a[i]!="/":
            s=s+a[i]
            i+=1
    return s


In [21]:
def extractSymptoms(text,listOfSymptoms):
    listOfSymptoms=[x.lower() for x in listOfSymptoms]
    bag=[]
    for item in listOfSymptoms:
        if item in text.lower():
            bag.append(item)
            
    return bag
    

In [22]:
listOfSymptoms=["high fever" , "weakness" ,"constipation" , "headache" , "poor apetite" , "stomach pain" , "fatigue" , "rash",
                "abdominal pain" ,"sweating" ,"muscle aches","head is aching","muscles are aching","sweat","rashes"
                
                "sore throat","runny nose","stuffy nose","cough","mild fever","fever","watery nose"
                
                "a yellow tinge to the skin","yellow skin ","yellowish skin","pale stools","a yellow tinge to the eye","yellow eye whites ","yellowish eye whites",
                "dark urine","yellow urine","dark yellow urine","yellowish urine",
                "itchiness","itching",
                "vomiting","vomit","abdominal pain","pain the the abdomen","pain in abdominal region",
                "weakness",
                "loss of apetite","apetite loss",
                
                "Sudden, high fever","sudden fever","high fever",
                "severe headaches","headaches",
                "abdominal pain","pain in the abdomen","pain in abdominal region",
                "weakness",
                "Pain behind the eyes","pain in eyes",
                "Severe joint and muscle pain","joint and muscle pain",
                "Fatigue",
                "vomiting","vomit",
                "mild bleeding","nose bleeding"

               
               
               ]

In [23]:
def appendPatch(prevSent,lst):
    for i in lst:
        if i not in prevSent:
            
            prevSent = prevSent + " , "+i+" , "
    prevSent = prevSent.rstrip(", ")
    return prevSent

In [24]:
import re
def clean_text(text):
    #text=text.lower()
    text=re.sub(r"i'm","i am",text)
    text=re.sub(r"I'm","I am",text)
    text=re.sub(r"he's","he is",text)
    text=re.sub(r"He's","He is",text)
    text=re.sub(r"she's","she is",text)
    text=re.sub(r"She's","She is",text)
    text=re.sub(r"that's","that is",text)
    text=re.sub(r"That's","That is",text)
    text=re.sub(r"what's","what is",text)
    text=re.sub(r"What's","What is",text)
    text=re.sub(r"where's","where is",text)
    text=re.sub(r"Where's","Where is",text)
    
    text=re.sub(r"\'ll"," will",text)
    text=re.sub(r"\'s"," is",text)
    text=re.sub(r"\'m"," am",text)
    
    text=re.sub(r"\'ve"," have",text)
    text=re.sub(r"\'re"," are",text)
    text=re.sub(r"\'d"," would",text)
    text=re.sub(r"won't"," will not",text)
    text=re.sub(r"can't"," can not",text)
    text=re.sub(r"[-()\"#$/?|.<>@:;+=,]","",text)
    return text

In [25]:
clean_text("I'm Ravi")

'I am Ravi'

In [36]:
user_data={}
bag=[]
print("Agent : "+"Hi !! I am your Medical Assistant . My name is MEDANT. Say stop to stop chatting")
while True:
    
    a=input( 'User : ' )
    if a == "stop":
        print("Quitting .. Have a nice day !!!")
        break

        
    a_clean=clean_text(a)
    #print(a_clean)
    a_1=a_clean.lower()
    
    
    
        
    if a_1 in ["hi","hi there","hello","hola","well hello there","well , hello there"]:
        print( "Agent : "+ " Hi . What's your name ?" )
        continue
    
    if a_1 in ["thanks","thank you","thankyou","thanks a lot","thankyou very much","thank you very much","thanks for the help"]:
        print( "Agent : "+ " No problem . What else can I do for you ?" )
        continue
    
    s=solve(a_clean)
    if s!="":
        user_data["name"]=s
        print("Agent : Hi "+ s + " ! How can I help you ..")
        continue
        
    bag.extend(extractSymptoms(a_1,listOfSymptoms))
    a_1 = appendPatch(a_1,bag)
    
    ls=a_1.strip().split()
    
    for i in ls :
        if i not in list(dic.keys()):
            a_1=a_1.replace(i," ")
            
    states_values = enc_model.predict( str_to_tokens(a_1 ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 
    deco=decoded_translation.strip('end')
    
    print( "Agent : "+ deco.strip().capitalize() )

    

Agent : Hi !! I am your Medical Assistant . My name is MEDANT. Say stop to stop chatting
User : Hi
Agent :  Hi . What's your name ?
User : Swastik Gupta
Agent : Hi Swastik ! How can I help you ..
User : I have been having itchiness and weakness for some days
Agent : I think that you may have dengue but i need more specifics apologies
User : My urine has turned yellowish
Agent : I think that you may have jaundice but i need more specifics
User : stop
Quitting .. Have a nice day !!!


In [ ]:
bag

In [ ]:
s

In [ ]:
user_data

### Sample login demo

In [ ]:
while True:
    
    print("Do you have an account .. (Y/n)")
    a=input().lower()
    if a == "n":
        print("Sign up to access our assistant ... ")
        print("Pick a username ..")
        us_name=input()
        print("Password please")
        password=input()
        if us_name not in list(users.keys()):
            users[us_name]=password
        else:
            print("Sorry but it is taken ")



    elif a == "y":
        print("Give your Username")
        us_name=input()
        print("Password")  
        password=input()
        if us_name not in list(users.keys()):
            print("Please sign up")
        elif users[us_name]==password:
            print("Validated Successfully")
            break
        elif users[us_name] != password:
            print("Wrong password")
        
    else:
        print("Typo .. ")
        